# Apache Pulsar - Concepts and Architecture: Messaging
[Apache Pulsar 2.7.1 - Concepts and Architecture: Messaging](https://pulsar.apache.org/docs/en/concepts-messaging/)

Pulsar is built on the publish-subscribe pattern (often abbreviated to pub-sub). In this pattern, producers publish messages to topics. Consumers subscribe to those topics, process incoming messages, and send an acknowledgement when processing is complete.\
Pulsar是建立在发布-订阅模式（通常缩写为pub-sub）。在此模式中，生产者将消息发布到主题。消费者订阅这些主题，处理传入消息，并在处理完成时发送确认。

When a subscription is created, Pulsar retains all messages, even if the consumer is disconnected. Retained messages are discarded only when a consumer acknowledges that those messages are processed successfully.\
创建订阅后，Pulsar保留所有消息，即使消费者断开连接。保留的消息只有在消费者确认这些消息已被成功处理时才会被丢弃。

__[Benefits of Pub/Sub Messaging](https://aws.amazon.com/cn/pub-sub-messaging/benefits/)__\
In modern cloud architecture, applications are decoupled into smaller, independent building blocks that are easier to develop, deploy and maintain. Publish/Subscribe (Pub/Sub) messaging provides instant event notifications for these distributed applications.\
在现代云架构中，应用程序被解耦成更小、更独立的构建块，更易于开发、部署和维护。发布/订阅(Pub/Sub)消息传递为这些分布式应用程序提供即时事件通知。

The Publish Subscribe model enables event-driven architectures and asynchronous parallel processing, while improving performance, reliability and scalability.\
发布订阅模型支持事件驱动的体系结构和异步并行处理，同时提高性能、可靠性和可伸缩性。

* __Eliminate Polling (消除轮询)__\
Pub/Sub messaging provides significant advantages to developers who build applications that rely on real-time events. Message topics allow instantaneous, push-based delivery, eliminating the need for message consumers to periodically check or “poll” for new information and updates. This promotes faster response time and reduces the delivery latency that can be particularly problematic in systems where delays cannot be tolerated.\
发布/订阅消息为那些构建依赖于实时事件的应用程序的开发者提供了显著的优势。消息主题允许即时的、基于推送的传递，消除了消息使用者定期检查或“轮询”新信息和更新的需要。这促进了更快的响应时间，并减少了交付延迟，在不能容忍延迟的系统中，交付延迟可能特别有问题。

* __Dynamic Targeting (动态目标)__\
Pub/Sub makes discovery of services easier, more natural and less error prone. Instead of maintaining a roster of peers that an application can send messages to, a publisher will simply post messages to a topic. Then, any interested party will subscribe its endpoint to the topic, and start receiving these messages. Subscribers can change, upgrade, multiply or disappear and the system dynamically adjusts.\
发布/订阅使得发现服务更容易，更自然，更不容易出错。发布者将简单地向主题发布消息，而不是维护应用程序可以向其发送消息的对等体名册。然后，任何感兴趣的一方将订阅其端点到主题，并开始接收这些消息。用户可以更改、升级、增加或消失，系统会动态调整。

## Messages

Messages are the basic "unit" of Pulsar. The following table lists the components of messages.

## Producers

A producer is a process that attaches to a topic and publishes messages to a Pulsar broker. The Pulsar broker process the messages.\
生产者是一个连接到主题并向Pulsar broker发布消息的过程。Pulsar broker处理消息。

### Send modes

Producers send messages to brokers synchronously (sync) or asynchronously (async).\
生产者以同步(sync)或异步(async)的方式向broker发送消息。

### Compression

You can compress messages published by producers during transportation. Pulsar currently supports the following types of compression:\
在传输过程中，您可以压缩由生产者发布的消息。Pulsar目前支持以下类型的压缩:
* LZ4
* ZLIB
* ZSTD
* SNAPPY

### Batching

When batching is enabled, the producer accumulates and sends a batch of messages in a single request. The batch size is defined by the maximum number of messages and the maximum publish latency. Therefore, the backlog size represents the total number of batches instead of the total number of messages.\
当启用批处理时，生产者在单个请求中累积并发送一批消息。批处理大小由最大消息数量和最大发布延迟定义。因此，待定大小表示的是批的总数，而不是消息的总数。

### Chunking

When you enable chunking, read the following instructions.\
当你启用分块时，请阅读以下说明。
* Batching and chunking cannot be enabled simultaneously. To enable chunking, you must disable batching in advance.\
不能同时启用批处理和分块。要启用分块，必须先禁用批处理功能。
* Chunking is only supported for persisted topics.\
仅对持久化主题支持分块。
* Chunking is only supported for the exclusive and failover subscription modes.\
仅在独占模式和故障转移订阅模式下支持分块。

When chunking is enabled (`chunkingEnabled = true`), if the message size is greater than the allowed maximum publish-payload size, the producer splits the original message into chunked messages and publishes them with chunked metadata to the broker separately and in order. At the broker side, the chunked messages are stored in the managed-ledger in the same way as that of ordinary messages. The only difference is that the consumer needs to buffer the chunked messages and combines them into the real message when all chunked messages have been collected. The chunked messages in the managed-ledger can be interwoven with ordinary messages. If producer fails to publish all the chunks of a message, the consumer can expire incomplete chunks if consumer fail to receive all chunks in expire time. By default, the expire time is set to one hour.\
当分块被启用时(`chunkingEnabled = true`)，如果消息大小大于允许的最大发布负载大小，生产者将原始消息分割成分块的消息，并将它们与分块的元数据分别并依次发布给broker。在broker端，分块消息以与普通消息相同的方式存储在托管分类帐（ledger）中。唯一的区别是，消费者需要缓冲分块的消息，并在收集了所有分块的消息后将它们组合到真正的消息中。管理分类帐中的分块信息可以与普通信息交织在一起。如果生产者未能发布消息的所有块，如果消费者未能在过期时间内接收到所有块，则消费者可以将不完整的块过期。缺省情况下，过期时间设置为1小时。
> __ledger__ [ˈledʒə(r)] <n.>\
a book in which a bank, a business, etc. records the money it has paid and received 收支总账；分类账簿；分户账簿

#### Handle chunked messages with one producer and one ordered consumer
使用一个生产者和一个订购的消费者处理分块消息

As shown in the following figure, when a topic has one producer which publishes large message payload in chunked messages along with regular non-chunked messages. The producer publishes message M1 in three chunks M1-C1, M1-C2 and M1-C3. The broker stores all the three chunked messages in the managed-ledger and dispatches to the ordered (exclusive/failover) consumer in the same order. The consumer buffers all the chunked messages in memory until it receives all the chunked messages, combines them into one message and then hands over the original message M1 to the client.\
如下图所示，当一个主题有一个生产者，它以分块的消息和常规的非分块的消息发布大型消息负载。生产者发布消息M1分为三个块M1-C1、M1-C2和M1-C3。broker将所有三个分块消息存储在托管分类帐中，并以相同的顺序分派给预订（独占/故障转移）的消费者。消费者将所有分块消息缓冲到内存中，直到它接收到所有分块消息，将它们组合成一条消息，然后将原始消息M1移交给client。

<img src = "https://pulsar.apache.org/docs/assets/chunking-01.png">

#### Handle chunked messages with multiple producers and one ordered consumer
处理多个生产者和一个订购消费者的分块消息

When multiple publishers publish chunked messages into a single topic, the broker stores all the chunked messages coming from different publishers in the same managed-ledger. As shown below, Producer 1 publishes message M1 in three chunks M1-C1, M1-C2 and M1-C3. Producer 2 publishes message M2 in three chunks M2-C1, M2-C2 and M2-C3. All chunked messages of the specific message are still in order but might not be consecutive in the managed-ledger. This brings some memory pressure to the consumer because the consumer keeps separate buffer for each large message to aggregate all chunks of the large message and combine them into one message.\
当多个生产者将分块消息发布到单个主题时，broker将来自不同生产者的所有分块消息存储在同一个托管分类账中。如下所示，生产者1发布消息M1分为三个块M1-C1、M1-C2和M1-C3。生产者2以三个块发布消息M2: M2-C1、M2-C2和M2-C3。特定消息的所有分块消息仍然是有序的，但在托管分类帐中可能不是连续的。这给消费者带来了一些内存压力，因为消费者为每个大消息保留单独的缓冲区，以聚合大消息的所有块并将它们组合为一条消息。

<img src = "https://pulsar.apache.org/docs/assets/chunking-02.png">

## Consumers

A consumer is a process that attaches to a topic via a subscription and then receives messages.\
消费者是一个通过订阅连接到主题，然后接收消息的过程。

## Topics

As in other pub-sub systems, topics in Pulsar are named channels for transmitting messages from producers to consumers.\
和其他pub子系统一样，Pulsar中的主题被命名为从生产者向消费者传递消息的通道。

## Subscriptions

A subscription is a named configuration rule that determines how messages are delivered to consumers. Four subscription modes are available in Pulsar: exclusive, shared, failover, and key_shared. These modes are illustrated in the figure below.\
订阅是一个指定的配置规则，它决定了如何将消息传递给使用者。Pulsar有四种订阅模式：exclusive、shared、failover和key_shared。这些模式如下图所示。

<img src = "https://pulsar.apache.org/docs/assets/pulsar-subscription-modes.png">

__Pub-Sub or Queuing__\
In Pulsar, you can use different subscriptions flexibly.\
在Pulsar，你可以灵活地使用不同的订阅。
* If you want to achieve traditional "fan-out pub-sub messaging" among consumers, specify a unique subscription name for each consumer. It is exclusive subscription mode.\
如果希望在消费者之间实现传统的“扇形发布-订阅消息传递”，请为每个消费者指定唯一的订阅名称。exclusive模式。
* If you want to achieve "message queuing" among consumers, share the same subscription name among multiple consumers(shared, failover, key_shared).\
如果希望在使用者之间实现“消息队列”，请在多个使用者之间共享相同的订阅名称(shared、failover、key_shared)。
* If you want to achieve both effects simultaneously, combine exclusive subscription mode with other subscription modes for consumers.\
如果想同时达到这两种效果，可以将exclusive模式与其他订阅模式结合起来，供消费者使用。

### Consumerless Subscriptions and Their Corresponding Modes

When a subscription has no consumers, its subscription mode is undefined. A subscription's mode is defined when a consumer connects to the subscription, and the mode can be changed by restarting all consumers with a different configuration.\
当订阅没有消费者时，它的订阅模式是未定义的。订阅模式是在使用者连接到订阅时定义的，可以通过使用不同的配置重新启动所有使用者来更改该模式。

### Exclusive

In exclusive mode, only a single consumer is allowed to attach to the subscription. If multiple consumers subscribe to a topic using the same subscription, an error occurs.\
在exclusive模式下，只允许一个消费者连接到订阅。如果多个消费者使用同一订阅去订阅一个主题，则会发生错误。

In the diagram below, only __Consumer A-0__ is allowed to consume messages.\
在下面的图表中，只有__消费者A-0__被允许使用消息。

<img src = "https://pulsar.apache.org/docs/assets/pulsar-exclusive-subscriptions.png">

Exclusive mode is the default subscription mode.\
exclusive模式是默认的订阅模式。

### Failover

In failover mode, multiple consumers can attach to the same subscription. A master consumer is picked for non-partitioned topic or each partition of partitioned topic and receives messages. When the master consumer disconnects, all (non-acknowledged and subsequent) messages are delivered to the next consumer in line.\
在failover模式中，多个消费者可以附加到同一订阅。为非分区主题或分区主题的每个分区选择一个主消费者，并接收消息。当主消费者断开连接时，所有（未确认的和后续的）消息将被传递给当行的下一个消费者。

For partitioned topics, broker will sort consumers by priority level and lexicographical order of consumer name. Then broker will try to evenly assigns topics to consumers with the highest priority level.\
对于分区主题，broker将按优先级级别和消费者名称的字典顺序对消费者进行排序。然后broker将尝试将主题平均分配给具有最高优先级的消费者。

For non-partitioned topic, broker will pick consumer in the order they subscribe to the non partitioned topic.\
对于非分区主题，broker将按照消费者订阅非分区主题的顺序选择消费者。

In the diagram below, __Consumer-B-0__ is the master consumer while __Consumer-B-1__ would be the next consumer in line to receive messages if __Consumer-B-0__ is disconnected.\
在下面的图表中，__消费者B-0__是主消费者，而如果__消费者B-0__断开连接，__消费者B-1__将是下一个排队接收消息的消费者。

<img src = "https://pulsar.apache.org/docs/assets/pulsar-failover-subscriptions.png">

### Shared

In shared or round robin mode, multiple consumers can attach to the same subscription. Messages are delivered in a round robin distribution across consumers, and any given message is delivered to only one consumer. When a consumer disconnects, all the messages that were sent to it and not acknowledged will be rescheduled for sending to the remaining consumers.\
在共享或轮询模式中，多个消费者可以连接到同一订阅。消息以轮询分发的方式跨消费者传递，任何给定的消息只传递给一个消费者。当一个消费者断开连接时，所有发送到它且未得到确认的消息将被重新安排发送到其他消费者。

In the diagram below, __Consumer-C-1__ and __Consumer-C-2__ are able to subscribe to the topic, but __Consumer-C-3__ and others could as well.\
在下面的图表中，__消费者C-1__和__消费者C-2__能够订阅主题，但__消费者C-3__和其他用户也可以。

<img src = "https://pulsar.apache.org/docs/assets/pulsar-shared-subscriptions.png">

__Limitations of shared mode__

When using shared mode, be aware that:\
当使用共享模式时，请注意:
* Message ordering is not guaranteed.\
消息排序没有保证。
* You cannot use cumulative acknowledgment with shared mode.\
您不能在共享模式下使用累积确认。

### Key_Shared

In Key_Shared mode, multiple consumers can attach to the same subscription. Messages are delivered in a distribution across consumers and message with same key or same ordering key are delivered to only one consumer. No matter how many times the message is re-delivered, it is delivered to the same consumer. When a consumer connected or disconnected will cause served consumer change for some key of message.\
在Key_Shared模式下，多个消费者可以连接到同一个订阅。消息在分发中跨消费者传递，具有相同键或相同订购键的消息只传递给一个消费者。无论消息被重新传递多少次，它都会被传递给相同的消费者。当消费者连接或断开时，将导致服务的消费者更改消息的某些键。

<img src = "https://pulsar.apache.org/docs/assets/pulsar-key-shared-subscriptions.png">

__Limitations of Key_Shared mode__

When you use Key_Shared mode, be aware that:\
当您使用Key_Shared模式时，请注意:
* You need to specify a key or orderingKey for messages.\
您需要为消息指定一个键或orderingKey。
* You cannot use cumulative acknowledgment with Key_Shared mode.
* Your producers should disable batching or use a key-based batch builder.

## Multi-topic subscriptions

When a consumer subscribes to a Pulsar topic, by default it subscribes to one specific topic, such as `persistent://public/default/my-topic`. As of Pulsar version 1.23.0-incubating, however, Pulsar consumers can simultaneously subscribe to multiple topics. You can define a list of topics in two ways:\
当一个消费者订阅一个Pulsar主题时，默认情况下它会订阅一个特定的主题，例如`persistent://public/default/my-topic`。然而，在Pulsar 1.23.0-incubating版本中，Pulsar消费者可以同时订阅多个主题。你可以用两种方式定义一个主题列表:
* On the basis of a regular expression (regex), for example `persistent://public/default/finance-.*`\
基于正则表达式(regex)，例如`persistent://public/default/finance-.*`
* By explicitly defining a list of topics\
通过显式定义一个主题列表

## Partitioned topics

Normal topics are served only by a single broker, which limits the maximum throughput of the topic. Partitioned topics are a special type of topic that are handled by multiple brokers, thus allowing for higher throughput.\
普通主题仅由单个broker提供服务，这限制了主题的最大吞吐量。分区主题是一种特殊类型的主题，由多个broker处理，因此允许更高的吞吐量。

A partitioned topic is actually implemented as N internal topics, where N is the number of partitions. When publishing messages to a partitioned topic, each message is routed to one of several brokers. The distribution of partitions across brokers is handled automatically by Pulsar.\
分区主题实际上是由N个内部主题实现的，其中N是分区的数量。在将消息发布到分区主题时，每个消息都被路由到多个broker中的一个。Pulsar自动处理跨broker的分区分布。

The diagram below illustrates this:\
下面的图表说明了这一点：
<img src = "https://pulsar.apache.org/docs/assets/partitioning.png">

The Topic1 topic has five partitions (P0 through P4) split across three brokers. Because there are more partitions than brokers, two brokers handle two partitions a piece, while the third handles only one (again, Pulsar handles this distribution of partitions automatically).\
Topic1主题有五个分区（P0到P4），它们在三个broker上分开。因为分区比broker多，所有两个broker处理两个分区，而第三个broker只处理一个分区（再次说明，Pulsar自动处理分区的分布）。

Messages for this topic are broadcast to two consumers. The [routing mode](https://pulsar.apache.org/docs/en/concepts-messaging/#routing-modes) determines each message should be published to which partition, while the [subscription mode](https://pulsar.apache.org/docs/en/concepts-messaging/#subscription-modes) determines which messages go to which consumers.\
此主题的消息被广播给两个消费者。路由模式确定应该将每个消息发布到哪个分区，而订阅模式确定哪些消息将发送到哪个消费者。

Decisions about routing and subscription modes can be made separately in most cases. In general, throughput concerns should guide partitioning/routing decisions while subscription decisions should be guided by application semantics.\
在大多数情况下，路由和订阅模式可以分别做出决定。通常，吞吐量应该指导分区/路由决策，而订阅决策应该由应用程序语义指导。

There is no difference between partitioned topics and normal topics in terms of how subscription modes work, as partitioning only determines what happens between when a message is published by a producer and processed and acknowledged by a consumer.\
就订阅模式的工作方式而言，分区主题和普通主题之间没有区别，因为分区只决定消息在由生产者发布和由消费者处理和确认之间发生了什么。

Partitioned topics need to be explicitly created via the [admin API](https://pulsar.apache.org/docs/en/admin-api-overview). The number of partitions can be specified when creating the topic.\
分区主题需要通过管理API显式创建。可以在创建主题时指定分区的数量。

### Routing modes

When publishing to partitioned topics, you must specify a routing mode. The routing mode determines which partition---that is, which internal topic---each message should be published to.\
在发布到分区主题时，必须指定路由模式。路由模式确定每个消息应该发布到哪个分区（即哪个内部主题）。

## Non-persistent topics

By default, Pulsar persistently stores all unacknowledged messages on multiple BookKeeper bookies (storage nodes). Data for messages on persistent topics can thus survive broker restarts and subscriber failover.\
默认情况下，Pulsar持续地将所有未确认的消息存储在多个BookKeeper bookie（存储节点）上。因此，关于持久主题的消息的数据可以在broker重启和订阅者故障转移中继续存在。

Pulsar also, however, supports non-persistent topics, which are topics on which messages are never persisted to disk and live only in memory. When using non-persistent delivery, killing a Pulsar broker or disconnecting a subscriber to a topic means that all in-transit messages are lost on that (non-persistent) topic, meaning that clients may see message loss.\
然而，Pulsar也支持非持久化主题，即消息永远不会持久化到磁盘上，而只存在于内存中。在使用非持久性传递时，终止Pulsar broker或断开订阅方与主题的连接意味着该（非持久性）主题上的所有传输中的消息都会丢失，这意味着client端可能会看到消息丢失。